# <center> Lab 3-2: Test Send CAN
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" />
### <center> Instructor: Dr. Jeremy Daily<br>Fall 2020

## Prerequisites
Login tip: Send a keepalive signal every 60 seconds from the connection menu of PuTTy.

This notebook must be run on a Linux machine with SocketCAN enabled.
```
sudo ip link add dev vcan1 type vcan
sudo ip link set vcan1 up
```

If you are running from PuTTy through Windows, then be sure to start the Jupyter server with the following commands:

```
jupyter notebook --ip cybertruck1.engr.colostate.edu --no-browser
```

or for the Beagle Bone Black connected by USB:
```
jupyter notebook --ip 192.168.7.2 --no-browser
```
Once these commands are successful, there will be a tokenize URL to paste into your browser. Note: this is not a secure connection, so be sure you are behind a firewall.


Be sure can-utils is installed:
```
sudo apt install can-utils
```

In [1]:
#!/usr/bin/python3
import socket
import struct

# Open a socket and bind to it from SocketCAN
sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)

#Change this interface to match your desired connection
interface = "vcan1"

# Bind to the interface
sock.bind((interface,))

# To match the socketCAN data structure, the following struct format can be used:

# Automatically detect the can frame format
try:
    # This is for the Beaglebone
    can_frame_format = "=lB3x8s"
    can_packet = struct.pack(can_frame_format, 0x18fef100 | socket.CAN_EFF_FLAG, 8, b'12345678' )
    print("Use this CAN Frame format for the rest of the notebooks: {}".format(can_frame_format))
except:
    # Use this format for x86-64, like cybertruck1.engr.colostate.edu
    can_frame_format = "=LB3x8s"
    can_packet = struct.pack(can_frame_format, 0x18fef100 | socket.CAN_EFF_FLAG, 8, b'12345678' )
    print("Use this CAN Frame format for the rest of the notebooks: {}".format(can_frame_format))



for i in range(5):
    # Define the CAN Arbitration ID
    can_id = 0x1FFFFFFF
    #Set the extended frame format bit.
    can_id |= socket.CAN_EFF_FLAG
    
    can_data = struct.pack("B",i) + b'eagr1234567890' 
    
    #The data length code cannot be larger than 8
    can_dlc = min(len(can_data),8)
    
    #Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
    print(can_packet)
    
    #Send out the CAN frame
    sock.send(can_packet)

AttributeError: module 'socket' has no attribute 'PF_CAN'

## Testing and Verification
Open another terminal on the server where the interface exists. From this terminal, enter the command:
```
candump any |grep FFFFFF
```
The output should look something like this when CAN messages are sent:
```
  vcan1  1FFFFFFF   [8]  00 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  01 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  02 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  03 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  04 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  05 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  06 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  07 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  08 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  09 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  0A 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  0B 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  0C 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  0D 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  0E 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  0F 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  10 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  11 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  12 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  13 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  14 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  15 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  16 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  17 65 61 67 72 31 32 33
  vcan1  1FFFFFFF   [8]  18 65 61 67 72 31 32 33

```


Now we have shown that we can send CAN data. Let's start looking at how we can use it for things like J1939.